## Create a graph from the pandas DataFrame

Let's start by creating a graph from a pandas DataFrame. In this exercise, you'll create a new bipartite graph by looping over the edgelist (which is a DataFrame object).

For simplicity's sake, in this graph construction procedure, any edge between a student and a forum node will be the 'last' edge (in time) that a student posted to a forum over the entire time span of the dataset, though there are ways to get around this.

Additionally, to shorten the runtime of the exercise, we have provided a sub-sampled version of the edge list as data. Explore it in the IPython Shell to familiarize yourself with it.

### Instructions
    - Instantiate a new Graph called G.
    - Add nodes from each of the partitions. Use the .add_nodes_from() method to do this. The two partitions are 'student' and 'forum'. To add nodes from the 'student' partition, for example, the arguments to .add_nodes_from() would be data['student'] and bipartite='student'.
    - Add in each edge along with the date the edge was created. To do this, use the .add_edge() method inside the loop, with the arguments d['student'], d['forum'], and date=d['date'].

In [ ]:
import networkx as nx

# Instantiate a new Graph: G
G = nx.Graph()

# Add nodes from each of the partitions
G.add_nodes_from(data['student'], bipartite='student')
G.add_nodes_from(data['forum'], bipartite='forum')

# Add in each edge along with the date the edge was created
for r, d in data.iterrows():
    G.add_edge(d['student'], d['forum'], date=d['date']) 

## Visualize the degree centrality distribution of the students projection

In this exercise, you will visualize the degree centrality distribution of the students projection. This is a recap of two previous concepts you've learned: degree centralities, and projections.

### Instructions
    - Get the nodes of the 'student' partition into a list called student_nodes.
        - Use a list comprehension to do this, iterating over all the nodes of G (including the metadata), and checking to see if the 'bipartite' keyword of d equals 'student'.
    - Create the students nodes projection as a graph called G_students. Use the nx.bipartite.projected_graph() function to do this. Be sure to specify the keyword argument nodes=student_nodes.
    - Calculate the degree centrality of G_students using nx.degree_centrality(). Store the result as dcs.
    - Plot the histogram of degree centrality values.

In [ ]:
# Import necessary modules
import matplotlib.pyplot as plt
import networkx as nx

# Get the student partition's nodes: student_nodes
student_nodes = [n for n, d in G.nodes(data=True) if d['bipartite'] == 'student']

# Create the students nodes projection as a graph: G_students
G_students = nx.bipartite.projected_graph(G, nodes=student_nodes)

# Calculate the degree centrality using nx.degree_centrality: dcs
dcs = nx.degree_centrality(G_students)

# Plot the histogram of degree centrality values
plt.hist(list(dcs.values()))
plt.yscale('log')  
plt.show() 

## Visualize the degree centrality distribution of the forums projection

This exercise is also to reinforce the concepts of degree centrality and projections. This time round, you'll plot the degree centrality distribution for the 'forum' projection. Follow the same steps as in the previous exercise!

### Instructions
    - Get the nodes of the 'forum' partition into a list called forum_nodes.
    - Create the forums nodes projection as a graph called G_forum.
    - Calculate the degree centrality of G_forum using nx.degree_centrality(). Store the result as dcs.
    - Plot the histogram of degree centrality values.

In [ ]:
# Import necessary modules
import matplotlib.pyplot as plt 
import networkx as nx

# Get the forums partition's nodes: forum_nodes
forum_nodes = [n for n, d in G.nodes(data=True) if d['bipartite'] == 'forum']

# Create the forum nodes projection as a graph: G_forum
G_forum = nx.bipartite.projected_graph(G, nodes=forum_nodes)

# Calculate the degree centrality using nx.degree_centrality: dcs
dcs = nx.degree_centrality(G_forum)

# Plot the histogram of degree centrality values
plt.hist(list(dcs.values()))
plt.yscale('log') 
plt.show()  

## Time filter on edges

You're now going to practice filtering the graph using a conditional as applied to the edges. This will help you gain practice and become comfortable with list comprehensions that contain conditionals.

To help you with the exercises, remember that you can import datetime objects from the datetime module. On the graph, the metadata has a date key that is paired with a datetime object as a value.

### Instructions
    - Instantiate a new graph called G_sub.
    - Add nodes from the original graph (including the node metadata), using the .add_nodes_from() method.
    - Add edges using a list comprehension with one conditional on the edge dates, that the date of the edge is earlier than 2004-05-16. To do this:
        - Use the .add_edges_from() method with a list comprehension as the argument.
        - The output expression of the list comprehension is (u, v, d). Iterate over all the edges of G and check whether d['date'] is less than datetime(2004, 5, 16).

In [ ]:
import networkx as nx
from datetime import datetime

# Instantiate a new graph: G_sub
G_sub = nx.Graph()

# Add nodes from the original graph
G_sub.add_nodes_from(G.nodes(data=True), bipartite='student')

# Add edges using a list comprehension with one conditional on the edge dates, that the date of the edge is earlier than 2004-05-16.
G_sub.add_edges_from([(u, v, d) for u, v, d in G.edges(data=True) if d['date'] < datetime(2004,5,16)])

## Visualize filtered graph using nxviz

Here, you'll visualize the filtered graph using a circos plot. The circos plot is a natural choice for this visualization, as you can use node grouping and coloring to visualize the partitions, while the circular layout preserves the aesthetics of the visualization.

### Instructions
    - Compute degree centrality scores of each node using the bipartite module degree centralities, but based on the degree centrality in the original graph.
        - Use the nx.bipartite.degree_centrality() function for this, with the arguments G and nodes=forum_nodes.
    - Create a new circos plot with nodes colored and grouped (parameters node_color_by and group_by) by their partition label ('bipartite'), and ordered (parameter sort_by) by their degree centrality ('dc') and display it.
        - To ensure that the nodes are visible when displayed, we have included the argument node_enc_kwargs={'radius': 10}.

In [ ]:
# Import necessary modules
from nxviz import circos
import networkx as nx
import matplotlib.pyplot as plt

# Compute degree centrality scores of each node
dcs = nx.bipartite.degree_centrality(G, nodes=forum_nodes)
for n, d in G_sub.nodes(data=True):
    G_sub.nodes[n]['dc'] = dcs[n]

# Create the circos plot: c
c = circos(G_sub, node_color_by="bipartite", group_by="bipartite", sort_by="dc", node_enc_kwargs={'radius': 5})

# Display the plot
plt.show() 

## Plot number of posts being made over time

Let's recap how you can plot evolving graph statistics from the graph data. First off, you will use the graph data to quantify the number of edges that show up within a chunking time window of td days, which is 2 days in the exercise below.

The datetime variables dayone and lastday have been provided for you.

### Instructions
    - Define a timedelta of 2 days using the timedelta() function and specifying an argument for the days parameter.
    - Inside the while loop:
        - Filter edges such that they are within the sliding time window. Use a list comprehension to do this, where the output expression is (u, v, d), the iterable is G.edges(data=True), and there are two conditions: if d['date'] is >= curr_day and < than curr_day + td.
        - Append the number of edges (use the len() function to help you calculate this) to n_posts.
        - Increment curr_day by the time delta td.
    - Make a plot of n_posts using plt.plot().

In [ ]:
# Import necessary modules
from datetime import timedelta  
import matplotlib.pyplot as plt

# Define current day and timedelta of 2 days
curr_day = dayone
td = timedelta(days=2)

# Initialize an empty list of posts by day
n_posts = []
while curr_day < lastday:
    if curr_day.day == 1:
        print(curr_day) 
    # Filter edges such that they are within the sliding time window: edges
    edges = [(u, v, d) for u, v, d in G.edges(data=True) if d['date'] >= curr_day and d['date'] < curr_day + td]
    
    # Append number of edges to the n_posts list
    n_posts.append(len(edges))
    # Increment the curr_day by the time delta
    curr_day += td
   
# Create the plot 
plt.plot(n_posts)  
plt.xlabel('Days elapsed')
plt.ylabel('Number of posts')
plt.show()  

## Extract the mean degree centrality day-by-day on the students partition

Here, you're going to see if the mean degree centrality over all nodes is correlated with the number of edges that are plotted over time. There might not necessarily be a strong correlation, and you'll take a look to see if that's the case.

### Instructions
    - Instantiate a new graph called G_sub containing a subset of edges.
    - Add nodes from G, including the node metadata.
    - Add in edges that fulfill the criteria, using the .add_edges_from() method.
    - Get the students projection G_student_sub from G_sub using the nx.bipartite.projected_graph() function.
    - Compute the degree centrality of the students projection using nx.degree_centrality() (don't use the bipartite version).
    - Append the mean degree centrality to the list mean_dcs. Be sure to convert dc.values() to a list first.
    - Hit 'Submit Answer' to view the plot!

In [ ]:
from datetime import datetime, timedelta
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# Initialize a new list: mean_dcs
mean_dcs = []
curr_day = dayone
td = timedelta(days=2)

while curr_day < lastday:
    if curr_day.day == 1:
        print(curr_day) 
    # Instantiate a new graph containing a subset of edges: G_sub
    G_sub = nx.Graph()
    # Add nodes from G
    G_sub.add_nodes_from(G.nodes(data=True))
    # Add in edges that fulfill the criteria
    G_sub.add_edges_from([(u, v, d) for u, v, d in G.edges(data=True) if d['date'] >= curr_day and d['date'] < curr_day + td])
    
    # Get the students projection
    G_student_sub = nx.bipartite.projected_graph(G_sub, nodes=student_nodes)
    # Compute the degree centrality of the students projection
    dc = nx.degree_centrality(G_student_sub)
    # Append mean degree centrality to the list mean_dcs
    mean_dcs.append(np.mean(list(dc.values())))
    # Increment the time
    curr_day += td
    
plt.plot(mean_dcs)
plt.xlabel('Time elapsed')
plt.ylabel('Degree centrality.')
plt.show()

## Find the most popular forums day-by-day: I

Great stuff! You're onto the final two exercises - which are really just one long exercise. These will be a good memory workout for your Python programming skills!

We're going to see how many forums took the title of "the most popular forum" on any given time window.

### Instructions
    - Instantiate a list to hold the list of most popular forums by day called most_popular_forums.
    - Instantiate a list to hold the degree centrality scores of the most popular forums called highest_dcs.
    - Instantiate new graph called G_sub and add in the nodes from the original graph G using the .add_nodes_from() method.
    - Add in edges from the original graph G that fulfill the criteria (which are exactly the same as in the previous exercise).

In [ ]:
# Import necessary modules
from datetime import timedelta
import networkx as nx
import matplotlib.pyplot as plt

# Instantiate a list to hold the list of most popular forums by day: most_popular_forums
most_popular_forums = []
# Instantiate a list to hold the degree centrality scores of the most popular forums: highest_dcs
highest_dcs = []
curr_day = dayone  
td = timedelta(days=1)  

while curr_day < lastday:  
    if curr_day.day == 1: 
        print(curr_day) 
    # Instantiate new graph: G_sub
    G_sub = nx.Graph()
    
    # Add in nodes from original graph G
    G_sub.add_nodes_from(G.nodes(data=True))
    
    # Add in edges from the original graph G that fulfill the criteria
    G_sub.add_edges_from([(u, v, d) for u, v, d in G.edges(data=True) if d['date'] >= curr_day and d['date'] < curr_day + td])
    
    # CODE CONTINUES ON NEXT EXERCISE
    curr_day += td

## Find the most popular forums day-by-day: II

Great work with the previous exercise - you had written code that created the time-series graph list. Now, you're going to finish that exercise - that is, you'll find out how many forums had the most popular forum score on a per-day basis!

One of the things you will be doing here is a "dictionary comprehension" to filter a dictionary. It is very similar to a list comprehension to filter a list, except the syntax looks like: {key: val for key, val in dict.items() if ...}. Keep that in mind!

### Instructions
    - Get the degree centrality using nx.bipartite.degree_centrality(), with G_sub and forum_nodes as arguments.
    - Filter the dictionary such that there's only forum degree centralities. The key: val pair in the output expression should be n, dc. Iterate over dc.items() and check if n is in forum_nodes.
    - Identify the most popular forum(s) - should be of highest degree centrality (max(forum_dcs.values())) and its DC value should not be zero.
    - Append the highest dc values to highest_dcs.
    - Create the plots!
        - Use a list comprehension for the first plot, in which you iterate over most_popular_forums (which is a list of lists) using forums as your iterator variable. The output expression should be the number of most popular forums, calculated using len().
        - For the second plot, use highest_dcs and plt.plot() to visualize the top degree centrality score

In [ ]:
# Import necessary modules
from datetime import timedelta
import networkx as nx
import matplotlib.pyplot as plt

most_popular_forums = []
highest_dcs = []
curr_day = dayone 
td = timedelta(days=1)  

while curr_day < lastday:  
    if curr_day.day == 1:  
        print(curr_day)  
    G_sub = nx.Graph()
    G_sub.add_nodes_from(G.nodes(data=True))   
    G_sub.add_edges_from([(u, v, d) for u, v, d in G.edges(data=True) if d['date'] >= curr_day and d['date'] < curr_day + td])
    
    # Get the degree centrality 
    dc = nx.bipartite.degree_centrality(G_sub, forum_nodes)
    # Filter the dictionary such that there's only forum degree centralities
    forum_dcs = {n:dc for n, dc in dc.items() if n in forum_nodes}
    # Identify the most popular forum(s) 
    most_popular_forum = [n for n, dc in forum_dcs.items() if dc == max(forum_dcs.values()) and dc != 0] 
    most_popular_forums.append(most_popular_forum) 
    # Store the highest dc values in highest_dcs
    highest_dcs.append(max(forum_dcs.values()))
    
    curr_day += td  
    
plt.figure(1) 
plt.plot([len(forums) for forums in most_popular_forums], color='blue', label='Forums')
plt.ylabel('Number of Most Popular Forums')
plt.show()

plt.figure(2)
plt.plot(highest_dcs, color='orange', label='DC Score')
plt.ylabel('Top Degree Centrality Score')
plt.show()